In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_D/Science/*'

In [3]:
file_list = []
out_file = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p:
        if 'B0445+123' in p:
            file_list.append(p)
            frame = p.split('_')[-3][:4]
            out_file.append('B0445_r'+ frame + '_cleaned.fits')

In [5]:
### This part is to create sky subtracted spectra. After that we can use those spectra for further analysis.
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[20, 550])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True)  #outfile=out_file[i], 
    #spec.szap(outfile=out_file[i], use_skymod=True)
    filled_data, mask = szap(spec.data)
    #print(filled_data.shape)
    wav_im = pf.open(p)[19].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    sky_im = pf.open(p)[14].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    ivar_im = pf.open(p)[13].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    #print(ivar_im.shape)
    sci_hdr = pf.Header()
    sci_hdr.append(('OBJECT', 'B0445+123'))
    sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
    sci_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
    sci_im_hdu = pf.PrimaryHDU(data=filled_data, header=sci_hdr)
    
    ivar_hdr = pf.Header()
    ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
    ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr)
    
    sky_hdr = pf.Header()
    sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
    sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr)
    
    wav_hdr = pf.Header()
    wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
    wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr)
    
    mask_hdr = pf.Header()
    mask_hdr.append(('IMTYPE', 'MASK'))
    cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
        'location from median image'
    mask_hdr.append('', cmt)
    mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr)
    
    hdul = pf.HDUList([sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
    hdul.writeto(out_file[i])


Loading file ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_D/Science/spec2d_r201213_0031-B0445+123_LRISr_2020Dec13T093939.802.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_D/Science/spec2d_r201213_0031-B0445+123_LRISr_2020Dec13T093939.802.fits (HDU=12)
The input dataset was trimmed
 xrange: 20:551.  yrange: 0:4096
Final data dimensions (x y): 531 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 531


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


Loading file ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_D/Science/spec2d_r201213_0032-B0445+123_LRISr_2020Dec13T102324.288.fits
--

In [ ]:
## median image
spec_list = []
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[20, 550])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True)
    spec_list.append(spec.data)
median_im = np.median(np.asarray(spec_list), axis=0)

In [ ]:
pf.PrimaryHDU(median_im).writeto('B0445_median_image.fits')

In [ ]:
sp_1 = pf.open('B0445_skysub_0033.fits')[0].data
sp_2 = pf.open('B0445_median_image.fits')[0].data

In [ ]:
sp_1[:, 85:120].shape

In [ ]:
%matplotlib notebook
count, bins, patch= plt.hist(sp_1[400:600, 85:120].flatten(), bins=500)
counts, bins1, patch = plt.hist(sp_2[400:600, 85:120].flatten(), bins=500)
#bins2= plt.hist(sp2.flatten(), bins=250)
#bins = plt.hist((sp2-sp1).flatten(), bins=200)
plt.yscale('log')

In [4]:
def szap(sp_1):

    sp_2 = pf.open('B0445_median_image.fits')[0].data
    mask = np.zeros(sp_1.shape, dtype=bool)
    #print(sp_1.shape)

    for k in range(0, sp_1.shape[0], 400):
        for i in range(0, sp_1.shape[1], 5):
            sp1 = sp_1[k:k+400, i:i+5]
            sp2 = sp_2[k:k+400, i:i+5]
            count1, bins1 = np.histogram(sp1.flatten(), bins=1000)
            count2, bins2 = np.histogram(sp2.flatten(), bins=1000)
            sm1 = 0
            sm2 = 0
            cnt1 = 0
            cnt2 = 0
            tot_pix =sp1.flatten().shape[0]

            for j,p in enumerate(count1):
                if p > cnt1:
                    cnt1 = p
                    max_bin1 = bins1[j+1]
                sm1 += p
                if sm1/tot_pix >= 0.8 :
                    border1 = 0.5*(bins1[j+1] + bins1[j])
                    break
            #print(border)
            for j,p in enumerate(count2):
                if p > cnt2:
                    cnt2 = p
                    max_bin2 = 0.5*(bins2[j+1] + bins2[j])
                sm2 += p
                if sm2/tot_pix >= 0.8 :
                    border2 = bins2[j+1]
                    break

            a = np.where(sp1>border1)
            a =np.transpose(a)

            if abs(border1 - max_bin1) < 50 :
                mp = 50    #/ abs(border1 - max_bin1)
            elif abs(border1 - max_bin1) < 100 :
                mp = 150
            elif abs(border1 - max_bin1) < 300:
                mp = 200
            else:
                mp = 400

            for j, p in enumerate(a):

                #print(sp2[p[0]][p[1]])
                if sp2[p[0]][p[1]] >0 :
                    #pass 
                    if sp1[p[0]][p[1]] > (sp2[p[0]][p[1]]+mp) :
                        mask[k+p[0]][i+p[1]] =1 
                else:
                    if sp1[p[0]][p[1]] > 3*border1 :
                        mask[k+p[0]][i+p[1]] =1
                        
    #print(mask)
    sp_1[mask] = sp_2[mask]
    
    return sp_1, mask.astype(int)

In [ ]:
sp_1[mask] = -50000     #sp_2[mask]

In [ ]:
sp_1[mask] = sp_2[mask]

In [ ]:
pf.PrimaryHDU(sp_1).writeto('B0445_33_fill.fits')

In [ ]:
sss = pf.open('B0445_29_fill.fits')

In [ ]:
sci_im = pf.open('B0445_29_fill.fits')[0].data
#wav_im = 
#ivar = 
#mask = 


In [ ]:
ss =pf.open(file_list[0])

In [ ]:
sss[0].header

In [ ]:
out_file

In [ ]:
cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
        'location from median image'

In [ ]:
cmt

In [ ]:
def f(u):
    
    return u**2, u**3

In [ ]:
a, b = f(10)

In [ ]:
a = pf.open('B0445_r0031_cleaned.fits')

In [ ]:
a.info()

In [ ]:
a[4].data

In [ ]:
ss[12].header